Bi Encoder

In [2]:
from query import query_qdrant
bi_encoder_retrival = query_qdrant(query_text = "By what factor does the proposed 3D representation in PhysX-Anything reduce the number of geometry tokens?")

[12/26/25 15:01:46] INFO     HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200  ]8;id=526031;file://d:\Narwal\mcp_rag\.venv\Lib\site-packages\httpx\_client.py\_client.py]8;;\:]8;id=240087;file://d:\Narwal\mcp_rag\.venv\Lib\site-packages\httpx\_client.py#1025\1025]8;;\
                             OK"                                                                                   

                    INFO     HTTP Request: GET                                                      ]8;id=877612;file://d:\Narwal\mcp_rag\.venv\Lib\site-packages\httpx\_client.py\_client.py]8;;\:]8;id=380001;file://d:\Narwal\mcp_rag\.venv\Lib\site-packages\httpx\_client.py#1025\1025]8;;\
                             https://07179cff-5524-4d69-b685-ff8e8896e747.us-east4-0.gcp.cloud.qdra                
                             nt.io:6333 "HTTP/1.1 200 OK"                                                          

[12/26/25 15:01:47] INFO     HTTP Request: POST                                                     ]8;id=905974;file://d:\Narwal\mcp_rag\.venv\Lib\site-packages\httpx\_client.py\_client.py]8;;\:]8;id=657572;file://d:\Narwal\mcp_rag\.venv\Lib\site-packages\httpx\_client.py#1025\1025]8;;\
                             https://07179cff-5524-4d69-b685-ff8e8896e747.us-east4-0.gcp.cloud.qdra                
                             nt.io:6333/collections/rag_mcp/points/query "HTTP/1.1 200 OK"                         

In [3]:
bi_encoder_retrival

[ScoredPoint(id='9f073b3b-9107-4aa9-923e-2604298c0395', version=11, score=0.8305408, payload={'text': '-\nsidering both geometry and all physical attributes.                                In to-\ntal, we collect 1,568 valid scores from 14 volunteers and\nnormalize the scores. The results show that the outputs of\nPhysX-Anything align much better with human preferences\nthan those of other methods, confirming its robust gener-\native performance in both geometry and physical proper-\n\n                                                                                                     6\n', 'metadata': {'file_path': 'D:\\Narwal\\mcp_rag\\data\\paper1.pdf', 'file_name': 'paper1.pdf', 'file_type': 'application/pdf', 'file_size': 4994615, 'creation_date': '2025-12-22', 'last_modified_date': '2025-12-22'}, 'chunk_index': 1, 'source_doc': 'dca54582-14cc-44a2-acaa-a22cbd7cdeb9'}, vector=None, shard_key=None, order_value=None),
 ScoredPoint(id='be308b15-3fff-4a3e-ac41-a95f93944955', version=1

 Cross-Encoder reranking

In [4]:

import os, requests
from dotenv import load_dotenv
load_dotenv()
VOYAGE_API_KEY = os.environ["VOYAGE_API_KEY"]
url = "https://api.voyageai.com/v1/rerank"

def rerank(query: str, chunks: list[str], top_k: int = 5):
    payload = {
        "model": "rerank-2.5",   # or "rerank-2.5"
        "query": query,
        "documents": chunks,
        "top_k": top_k,
    }
    headers = {"Authorization": f"Bearer {VOYAGE_API_KEY}"}
    r = requests.post(url, json=payload, headers=headers)
    r.raise_for_status()
    return r.json()
query = "What is Model Context Protocol?"

chunks = [
    "Model Context Protocol (MCP) is a standard that allows large language models to call tools using a structured interface.",
    "FastAPI is a modern Python web framework used to build APIs quickly and efficiently.",
    "MCP enables LLMs to interact with external systems through JSON-RPC style messages.",
    "Qdrant is a vector database designed for similarity search and retrieval.",
    "The protocol defines how models expose tools and how clients invoke them."
]
result = rerank(query, chunks, top_k=3)


In [13]:
cross_encoder_chunks =[i.payload["text"] for i in bi_encoder_retrival]
cross_encoder_metadata = [i.payload["metadata"] for i in bi_encoder_retrival]
cross_encoder_chunks

['-\nsidering both geometry and all physical attributes.                                In to-\ntal, we collect 1,568 valid scores from 14 volunteers and\nnormalize the scores. The results show that the outputs of\nPhysX-Anything align much better with human preferences\nthan those of other methods, confirming its robust gener-\native performance in both geometry and physical proper-\n\n                                                                                                     6\n',
 '\n    Input    PhysX-Anything (Voxel)    PhysX-Anything (Index)    PhysX-Anything (Ours)\n             Geometry    Kinematic     Geometry    Kinematic     Geometry    Kinematic\n\n              Question: Find Dimension: 90*60*100                              Dimension: 90*60*100                                        Dimension: 90*60*100\n              a wheel of the shopping cart made of rubber                      Question: Find a wheel of the shopping cart made of rubber  Question: Find a whee

In [14]:
question  = "By what factor does the proposed 3D representation in PhysX-Anything reduce the number of geometry tokens?"
cross_encoder_rerank = rerank(question, cross_encoder_chunks, top_k=3)
cross_encoder_rerank

{'object': 'list',
 'data': [{'relevance_score': 0.8359375, 'index': 8},
  {'relevance_score': 0.8203125, 'index': 4},
  {'relevance_score': 0.7890625, 'index': 5}],
 'model': 'rerank-2.5',
 'usage': {'total_tokens': 6893}}

Late Interaction reranking

In [19]:
cross_encoder_rerank_chunks = []
for i in cross_encoder_rerank["data"]:
    index = i.get("index")
    cross_encoder_rerank_chunks.append(cross_encoder_chunks[index])

cross_encoder_rerank_chunks

['\n                 Avg Token of Parts=177450                           Avg Token of Part=118490\nv    ⎵      0 .       0 2  ⎵ 0 .  7 1     ⎵     0  .    0  5         v     ⎵    8  ⎵  3     0       ⎵    1    2\n     ⎵                     ⎵              ⎵                     Quantization\nv           0 .       0 3    0 .  7 2           0  .    0  2         v     ⎵    1  3  ⎵     3       1    ⎵    1     2\nMesh ⎵                     ⎵              ⎵                          v     ⎵ ⎵        ⎵ ⎵                ⎵ ⎵\nv           0 .       0 3    0 .  7 2           0  .    0  3                    1  3        3       1         1     2\n     ⎵                     ⎵              ⎵                                ⎵          ⎵                  ⎵\nOriginal                                                             v\nv           0 .       0 3    0 .  7 2           0  .    0  4                    1  3        3       1         1     2\n     ⎵                     ⎵              ⎵                    

In [1]:
from transformers import AutoTokenizer, AutoModel
import torch
import torch.nn.functional as F

# Load ColBERTv2 model & tokenizer
colbert_tokenizer = AutoTokenizer.from_pretrained("colbert-ir/colbertv2.0")
colbert_model = AutoModel.from_pretrained("colbert-ir/colbertv2.0")
colbert_model.eval()

def get_token_embeddings(text, tokenizer, model, max_length=128):
    # Get token-level embeddings, ignore [CLS] and [SEP]
    inputs = tokenizer(text, return_tensors='pt', truncation=True, max_length=max_length)
    with torch.no_grad():
        outputs = model(**inputs)

    # outputs.last_hidden_state: (1, seq_len, hidden_dim)
    input_ids = inputs['input_ids'][0]
    keep_indices = (input_ids != tokenizer.cls_token_id) & (input_ids != tokenizer.sep_token_id)

    return outputs.last_hidden_state[0][keep_indices]  # (filtered_seq_len, hidden_dim)

def colbert_score(query_emb, doc_emb):
    # query_emb: (m, d), doc_emb: (n, d)
    sim = F.cosine_similarity(query_emb.unsqueeze(1), doc_emb.unsqueeze(0), dim=2)  # (m, n)
    max_sim, _ = sim.max(dim=1)  # (m,)
    return max_sim.sum().item()

def rerank_with_late_interaction(query, chunks, top_k=5):
    # Precompute query embeddings once
    query_emb = get_token_embeddings(query, colbert_tokenizer, colbert_model)

    scores = []
    for i, ch in enumerate(chunks):
        doc_emb = get_token_embeddings(ch, colbert_tokenizer, colbert_model)
        score = colbert_score(query_emb, doc_emb)
        scores.append((score, i))  # keep index so you can map back to metadata

    scores.sort(reverse=True)  # highest score first
    top = scores[:top_k]

    # return list of dicts with index + score (index maps to original chunks list)
    return [{"index": i, "late_interaction_score": float(s)} for (s, i) in top]


d:\Narwal\mcp_rag\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
d:\Narwal\mcp_rag\.venv\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\sriha\.cache\huggingface\hub\models--colbert-ir--colbertv2.0. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this ar

In [7]:
late_interaction_rank  = rerank_with_late_interaction(question, cross_encoder_chunks, top_k=3)
late_interaction_rank

[{'index': 3, 'late_interaction_score': 16.465396881103516},
 {'index': 0, 'late_interaction_score': 14.276972770690918},
 {'index': 2, 'late_interaction_score': 13.568941116333008}]

In [20]:
late_interaction_rerank_chunks = []
for i in late_interaction_rank:
    index = i.get("index")
    late_interaction_rerank_chunks.append(cross_encoder_chunks[index])
late_interaction_rerank_chunks

[' for\n• downstream applications in simulation and embodied AI.\n We propose a unified VLM-based generative pipeline to-\n gether with a    novel physical 3D representation. Our\n representation compresses geometry tokens at a high rate\n while preserving explicit geometric structure, and avoids\n• introducing any special tokens during fine-tuning.\n We construct a new physically grounded 3D dataset,\n PhysX-Mobility, which enriches the category diversity\n of existing physical 3D datasets by over  2×, including\n over 2K common real-world objects such as cameras, cof-\n• fee machines, and staplers.\n Through comprehensive evaluations on PhysX-Mobility\n and in-the-wild images, we demonstrate the strong gen-\n erative quality  and robust generalization of      PhysX-\n Anything. Furthermore, we validate the feasibility of di-\n rectly deploying our sim-ready assets in simulation envi-\n ronments, thereby empowering downstream applications\n such as embodied AI and robotic manipulation

In [21]:
from openai import OpenAI

client = OpenAI()

def gpt_rag_answer(query: str, chunks: list[str], model: str = "gpt-4o-mini") -> str:
    """
    Inputs:
      - query: user question (string)
      - chunks: list of retrieved/reranked text chunks (list[str])

    Output:
      - final answer string from GPT-4o-mini
    """
    context = "\n\n---\n\n".join(chunks)

    prompt = f"""Use ONLY the provided chunks to answer the question.

CHUNKS:
{context}

QUESTION: {query}

Rules:
- If the answer is not in the chunks, say: "Not found in provided context."
- Be specific and grounded in the chunks.
"""

    resp = client.chat.completions.create(
        model=model,
        messages=[{"role": "user", "content": prompt}],
        temperature=0.2,
    )
    return resp.choices[0].message.content.strip()


In [22]:
gpt_rag_answer(question, cross_encoder_rerank_chunks)

[12/26/25 15:36:00] INFO     HTTP Request: POST https://api.openai.com/v1/chat/completions          ]8;id=654182;file://d:\Narwal\mcp_rag\.venv\Lib\site-packages\httpx\_client.py\_client.py]8;;\:]8;id=250035;file://d:\Narwal\mcp_rag\.venv\Lib\site-packages\httpx\_client.py#1025\1025]8;;\
                             "HTTP/1.1 200 OK"                                                                     

'The proposed 3D representation in PhysX-Anything reduces the number of geometry tokens by a factor of 193×.'